In [ ]:
# === IMPORTS ===
import os
import json
import trimesh
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, MultiLineString, Point
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import numpy as np

# === AUTO GPU/CPU ===
try:
    import cupy as xp
    print("✅ Using GPU (CuPy)")
except ImportError:
    import numpy as xp
    print("⚠️ Using CPU (NumPy)")

# === LOAD STL ===
mesh = trimesh.load_mesh("repaired_print_demo90deg.stl")
mesh.apply_translation(-mesh.centroid)

# === AUTO PARAMETERS BASED ON MESH ===
height = mesh.bounds[1][2] - mesh.bounds[0][2]
width = mesh.bounds[1][0] - mesh.bounds[0][0]

nozzle_diameter = 0.4
layer_height = max(0.05, min(0.1, height / 200))
wall_count = max(2, int(width / 5))
infill_spacing = max(0.5, width / 20)
extrusion_multiplier = 0.045
travel_speed = 150
print_speed = 60

# === LOAD infill_patterns.json ===
if not os.path.exists("infill_patterns.json"):
    default_patterns = [
        {"type": "grid", "direction": "none"},
        {"type": "zigzag", "direction": "horizontal"},
        {"type": "zigzag", "direction": "vertical"},
        {"type": "zigzag", "direction": "diagonal"},
        {"type": "lines", "direction": "horizontal"},
        {"type": "none", "direction": "none"}
    ]
    with open("infill_patterns.json", "w") as f:
        json.dump(default_patterns, f, indent=2)

with open("infill_patterns.json") as f:
    infill_configs = json.load(f)

# === CALCULATE SCORE FUNCTION ===
def calculate_score(positions_by_layer, mesh, xp):
    valid_layers = [xp.array(layer) for layer in positions_by_layer if len(layer) > 0]
    if not valid_layers:
        return float("inf")
    all_points = xp.concatenate(valid_layers)
    centroid_diff = xp.linalg.norm(xp.mean(all_points, axis=0) - xp.asarray(mesh.centroid))
    spread_score = xp.std(all_points[:, 0]) + xp.std(all_points[:, 1]) + xp.std(all_points[:, 2])
    return centroid_diff + (1.0 / (spread_score + 1e-6))

# === INFILL GENERATOR FUNCTIONS ===

def generate_grid(poly, spacing, direction, layer_z, positions):
    minx, miny, maxx, maxy = poly.bounds
    x = minx
    while x <= maxx:
        line = LineString([(x, miny), (x, maxy)])
        clipped = line.intersection(poly)
        lines = [clipped] if isinstance(clipped, LineString) else list(clipped.geoms) if isinstance(clipped, MultiLineString) else []
        for ln in lines:
            for x0, y0 in ln.coords:
                positions.append([x0, y0, layer_z])
        x += spacing

    if direction == "cross":
        y = miny
        while y <= maxy:
            line = LineString([(minx, y), (maxx, y)])
            clipped = line.intersection(poly)
            lines = [clipped] if isinstance(clipped, LineString) else list(clipped.geoms) if isinstance(clipped, MultiLineString) else []
            for ln in lines:
                for x0, y0 in ln.coords:
                    positions.append([x0, y0, layer_z])
            y += spacing

def generate_zigzag(poly, spacing, direction, layer_z, positions):
    minx, miny, maxx, maxy = poly.bounds
    flip = 1
    if direction == "horizontal":
        y = miny
        while y <= maxy:
            line = LineString([(minx, y), (maxx, y)]) if flip == 1 else LineString([(maxx, y), (minx, y)])
            clipped = line.intersection(poly)
            lines = [clipped] if isinstance(clipped, LineString) else list(clipped.geoms) if isinstance(clipped, MultiLineString) else []
            for ln in lines:
                for x0, y0 in ln.coords:
                    positions.append([x0, y0, layer_z])
            y += spacing
            flip *= -1
    elif direction == "vertical":
        x = minx
        while x <= maxx:
            line = LineString([(x, miny), (x, maxy)]) if flip == 1 else LineString([(x, maxy), (x, miny)])
            clipped = line.intersection(poly)
            lines = [clipped] if isinstance(clipped, LineString) else list(clipped.geoms) if isinstance(clipped, MultiLineString) else []
            for ln in lines:
                for x0, y0 in ln.coords:
                    positions.append([x0, y0, layer_z])
            x += spacing
            flip *= -1
    elif direction == "diagonal":
        step = spacing / xp.sqrt(2)
        offset = minx - (maxy - miny)
        while offset < maxx:
            line = LineString([(offset, miny), (offset + (maxy - miny), maxy)])
            clipped = line.intersection(poly)
            lines = [clipped] if isinstance(clipped, LineString) else list(clipped.geoms) if isinstance(clipped, MultiLineString) else []
            for ln in lines:
                for x0, y0 in ln.coords:
                    positions.append([x0, y0, layer_z])
            offset += step
    elif direction == "inverse-diagonal":
        step = spacing / xp.sqrt(2)
        offset = maxx + (maxy - miny)
        while offset > minx:
            line = LineString([(offset, miny), (offset - (maxy - miny), maxy)])
            clipped = line.intersection(poly)
            lines = [clipped] if isinstance(clipped, LineString) else list(clipped.geoms) if isinstance(clipped, MultiLineString) else []
            for ln in lines:
                for x0, y0 in ln.coords:
                    positions.append([x0, y0, layer_z])
            offset -= step
    elif direction == "spiral":
        # Add spiral logic for zigzag (clockwise)
        center_x = (minx + maxx) / 2
        center_y = (miny + maxy) / 2
        angle = 0
        radius = 0
        while radius < maxx - minx:
            x = center_x + radius * np.cos(angle)
            y = center_y + radius * np.sin(angle)
            if poly.contains(Point(x, y)):
                positions.append([x, y, layer_z])
            angle += np.pi / 10
            radius += spacing

def generate_concentric(poly, spacing, direction, layer_z, positions):
    minx, miny, maxx, maxy = poly.bounds
    radius = np.sqrt((maxx - minx)**2 + (maxy - miny)**2) / 2
    center_x = (minx + maxx) / 2
    center_y = (miny + maxy) / 2
    
    # Inward: create circles that get smaller
    if direction == "inward":
        while radius > 0:
            for angle in np.linspace(0, 2 * np.pi, 100):
                x = center_x + radius * np.cos(angle)
                y = center_y + radius * np.sin(angle)
                if poly.contains(Point(x, y)):
                    positions.append([x, y, layer_z])
            radius -= spacing

    # Outward: create circles that get larger
    elif direction == "outward":
        while radius < maxx - minx:
            for angle in np.linspace(0, 2 * np.pi, 100):
                x = center_x + radius * np.cos(angle)
                y = center_y + radius * np.sin(angle)
                if poly.contains(Point(x, y)):
                    positions.append([x, y, layer_z])
            radius += spacing

    # Spiral: outward spiral
    elif direction == "spiral":
        angle = 0
        while radius < maxx - minx:
            x = center_x + radius * np.cos(angle)
            y = center_y + radius * np.sin(angle)
            if poly.contains(Point(x, y)):
                positions.append([x, y, layer_z])
            angle += np.pi / 10
            radius += spacing

# === INFILL MAP ===
infill_generators = {
    "grid": generate_grid,
    "zigzag": generate_zigzag,
    "lines": generate_zigzag,
    "none": lambda *args, **kwargs: None,
    "concentric": generate_concentric,
    "spiral": generate_zigzag,  # Assuming spiral could also use zigzag for simplicity
    "honeycomb": lambda *args, **kwargs: None,  # Implement honeycomb as needed
    "gyroid": lambda *args, **kwargs: None,  # Placeholder for gyroid
    "hilbert": lambda *args, **kwargs: None,  # Placeholder for Hilbert curve
}

# === SLICE LOOP AND FIND BEST INFILL ===
best_match = None
best_score = float("inf")
best_result = None

for config in infill_configs:
    infill_type = config["type"]
    infill_direction = config["direction"]
    z = layer_height
    positions_by_layer = []
    while z < mesh.bounds[1][2]:
        section = mesh.section(plane_origin=[0, 0, z], plane_normal=[0, 0, 1])
        if section is None or section.is_empty:
            z += layer_height
            continue
        paths_2d, _ = section.to_2D()
        polygons = [p for p in paths_2d.polygons_full if p.exterior and len(p.exterior.coords) >= 3]
        if not polygons:
            z += layer_height
            continue
        layer_positions = []
        for poly in polygons:
            if not poly.is_valid or poly.is_empty:
                continue
            shell = poly
            for wall in range(wall_count):
                sub_polys = [shell] if isinstance(shell, Polygon) else list(shell.geoms)
                for sp in sub_polys:
                    if sp.is_empty or not sp.exterior:
                        continue
                    coords = list(sp.exterior.coords)
                    for x, y in coords:
                        layer_positions.append([x, y, z])
                if shell.area < nozzle_diameter ** 2:
                    break
                shell = shell.buffer(-nozzle_diameter)
            # add infill
            infill_fn = infill_generators.get(infill_type, lambda *a, **kw: None)
            infill_fn(poly, infill_spacing, infill_direction, z, layer_positions)
        positions_by_layer.append(layer_positions)
        z += layer_height
    score = calculate_score(positions_by_layer, mesh, xp)
    if score < best_score:
        best_score = score
        best_result = positions_by_layer
        best_match = (infill_type, infill_direction)

# === PLOT RESULT 3D VIEW ===
positions_by_layer = best_result
infill_type, infill_direction = best_match
print(f"✅ Best match: {infill_type} / {infill_direction}")

verts = mesh.vertices
faces = mesh.faces

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'scene'}]],
                    subplot_titles=("STL Model", "G-code Preview"))

fig.add_trace(go.Mesh3d(
    x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
    i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
    color='lightblue', opacity=0.5,
    flatshading=True
), row=1, col=1)

for i, layer in enumerate(positions_by_layer):
    arr = xp.asnumpy(xp.array(layer))
    if len(arr) > 1:
        fig.add_trace(go.Scatter3d(
            x=arr[:, 0], y=arr[:, 1], z=arr[:, 2],
            mode='lines',
            line=dict(width=2, color=i, colorscale='Viridis'),
            name=f"Layer {i}"
        ), row=1, col=2)

fig.update_layout(
    title="STL vs G-code Preview",
    scene=dict(aspectmode='data', domain=dict(x=[0, 0.5])),
    scene2=dict(aspectmode='data', domain=dict(x=[0.5, 1.0])),
    uirevision='linked',
    margin=dict(l=0, r=0, t=50, b=0)
)
fig.show()

# === 2D VIEWER ===
def plot_layer(n):
    if n < len(positions_by_layer):
        layer = xp.asnumpy(xp.array(positions_by_layer[n]))
        if len(layer) == 0:
            print("No data for this layer.")
            return
        plt.figure(figsize=(6, 6))
        plt.plot(layer[:, 0], layer[:, 1], linewidth=0.8, color="royalblue")
        plt.title(f"2D View - Layer {n} ({infill_type} / {infill_direction})")
        plt.xlabel("X (mm)")
        plt.ylabel("Y (mm)")
        plt.axis("equal")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

layer_slider = widgets.IntSlider(min=0, max=len(positions_by_layer)-1, step=1, description='Layer')
widgets.interact(plot_layer, n=layer_slider)

✅ Using GPU (CuPy)


NameError: name 'Point' is not defined

In [ ]:
pip install --upgrade -r requirements.txt